In [3]:
import math
import numpy as np
import pandas as pd
import os

In [ ]:


def getcrossminlevel(dataframes1, dataframes2):
    # Levels: y: 3, m:2, d: 1, h:0, error: -1
    return min(getminlevel(dataframes1), getminlevel(dataframes2))


def getminlevel(dataframes):
    for k in range(4):
        if not math.isnan(dataframes[k].iloc[0, 0]):
            return k

    return -1


def cropdates(dataframes1, dataframes2):
    min1 = dataframes1.iloc[0, 0]
    min2 = dataframes2.iloc[0, 0]
    max1 = dataframes1.iloc[-1, 0]
    max2 = dataframes2.iloc[-1, 0]

    df_1 = dataframes1.copy()
    df_2 = dataframes2.copy()
    if min1 < min2:
        df_1 = dataframes1[dataframes1[0] >= min2]
    else:
        df_2 = dataframes2[dataframes2[0] >= min1]

    if max1 > max2:
        df_1 = df_1[df_1[0] <= max2]
    else:
        df_2 = df_2[df_2[0] <= max1]

    return df_1, df_2


def combinedf(df_1, df_2):
    # check if works
    df = pd.merge(df_1, df_2, on=[0])
    return df.dropna()


directory = './'

directory = os.fsencode(directory)

dataframes_list = []
counter = 0
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".csv"):
        df_h = pd.read_csv(filename, sep=';', usecols=[0, 1])
        df_h[0] = pd.to_datetime(df_h[0], format='%Y-%m-%dT%h')
        df_d = pd.read_csv(filename, sep=';', usecols=[2, 3])
        df_d[0] = pd.to_datetime(df_d[0], format='%Y-%m-%d')
        df_m = pd.read_csv(filename, sep=';', usecols=[4, 5])
        df_m[0] = pd.to_datetime(df_m[0], format='%Y-%m')
        df_y = pd.read_csv(filename, sep=';', usecols=[6, 7])
        df_y[0] = pd.to_datetime(df_y[0], format='%Y')
        dataframes_list[counter] = [df_h, df_d, df_m, df_y, filename]
        counter += 1


corrMatrix = np.zeroes(counter, counter)
for i in range(counter):
    for j in range(i+1, counter):
        level = getcrossminlevel(dataframes_list[i], dataframes_list[j])
        if level == -1:
            continue
        # Levels: y: 3, m:2, d: 1, h:0, error: -1
        dframe1, dframe2 = cropdates(dataframes_list[i][level], dataframes_list[j][level])
        df_for_corr = combinedf(dframe1, dframe2)
        df_for_corr[0] = df_for_corr[0].apply(lambda x: x.value)
        corrMatrix[i, j] = df_for_corr[1].corr(df_for_corr[2])


np.savetxt('correlationMatrix.csv', corrMatrix, delimiter=';')